To access a dataframe from SQL,Spark gives you two options.
* create a temporary view
* create a global view.

Spark.sql in azure databricks give the ability 
* to put the output data into a dataframe, and then to use DataFrame API
* to use a variable in a SQL statement

A temporary view is only valid within a Spark session, so it's not available in another notebook.
if you detach the notebook from the cluster and reattach it, this view is not going to be available.

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName('view').setMaster('local')
sc = SparkContext(conf = conf)
spark = SparkSession(sc)

In [4]:
race_result_df = spark.read.parquet(r'E:\unused\Udemy\Formula1_Project_Solutions\Formula1_Project_Solutions\raw\raw\presentation\race_result')
race_result_df.show(2)

+---------+-----------------+-------------------+----------------+--------------+-------------+------------------+-------+----+-----------+---------+------+--------+
|race_year|        race_name|          race_date|circuit_location|   driver_name|driver_number|driver_nationality|   team|grid|fastest_lap|race_time|points|position|
+---------+-----------------+-------------------+----------------+--------------+-------------+------------------+-------+----+-----------+---------+------+--------+
|     1974|German Grand Prix|1974-08-04 00:00:00|         Nürburg|Clay Regazzoni|         null|             Swiss|Ferrari|   2|       null|1:41:35.0|   9.0|       1|
|     1974|German Grand Prix|1974-08-04 00:00:00|         Nürburg|Jody Scheckter|         null|     South African|Tyrrell|   4|       null|    +50.7|   6.0|       2|
+---------+-----------------+-------------------+----------------+--------------+-------------+------------------+-------+----+-----------+---------+------+--------+
only

In [5]:
# race_result_df.createTempView('vw_race_result')
race_result_df.createOrReplaceTempView('vw_race_result')
df = spark.sql('SELECT * FROM vw_race_result WHERE race_year = 2020')
df.show(2)

+---------+------------------+-------------------+----------------+---------------+-------------+------------------+--------+----+-----------+-----------+------+--------+
|race_year|         race_name|          race_date|circuit_location|    driver_name|driver_number|driver_nationality|    team|grid|fastest_lap|  race_time|points|position|
+---------+------------------+-------------------+----------------+---------------+-------------+------------------+--------+----+-----------+-----------+------+--------+
|     2020|Russian Grand Prix|2020-09-27 00:00:00|           Sochi|Valtteri Bottas|           77|           Finnish|Mercedes|   3|         51|1:34:00.364|  26.0|       1|
|     2020|Russian Grand Prix|2020-09-27 00:00:00|           Sochi| Max Verstappen|           33|             Dutch|Red Bull|   2|         53|     +7.729|  18.0|       2|
+---------+------------------+-------------------+----------------+---------------+-------------+------------------+--------+----+-----------+---

In [10]:
race_result_df.createOrReplaceGlobalTempView('gvw_race_result')
# df = spark.sql('SELECT * FROM gvw_race_result WHERE race_year = 2020')
#when the above statement is excecuted it sows an error msg "Table or view not found"
# when global views are created it is stored under global_temp database
df = spark.sql('SELECT * FROM global_temp.gvw_race_result WHERE race_year = 2020')
df.show(2)

+---------+------------------+-------------------+----------------+---------------+-------------+------------------+--------+----+-----------+-----------+------+--------+
|race_year|         race_name|          race_date|circuit_location|    driver_name|driver_number|driver_nationality|    team|grid|fastest_lap|  race_time|points|position|
+---------+------------------+-------------------+----------------+---------------+-------------+------------------+--------+----+-----------+-----------+------+--------+
|     2020|Russian Grand Prix|2020-09-27 00:00:00|           Sochi|Valtteri Bottas|           77|           Finnish|Mercedes|   3|         51|1:34:00.364|  26.0|       1|
|     2020|Russian Grand Prix|2020-09-27 00:00:00|           Sochi| Max Verstappen|           33|             Dutch|Red Bull|   2|         53|     +7.729|  18.0|       2|
+---------+------------------+-------------------+----------------+---------------+-------------+------------------+--------+----+-----------+---